In [1]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import re
from textblob import TextBlob 
import gender_guesser.detector as gender

In [2]:
''' 
[Data Format Headers]
attendance
clarityColor
easyColor
helpColor
helpCount
id
notHelpCount
onlineClass
pName
quality
rClarity
rClass
rComments
rDate
rEasy
rEasyString
rErrorMsg
rHelpful
rInterest
rOverall
rOverallString
rStatus
rTextBookUse
rWouldTakeAgain
sId
takenForCredit
teacher
teacherGrade
teacherRatingTags
unUsefulGrouping
usefulGrouping
'''


' \n[Data Format Headers]\nattendance\nclarityColor\neasyColor\nhelpColor\nhelpCount\nid\nnotHelpCount\nonlineClass\npName\nquality\nrClarity\nrClass\nrComments\nrDate\nrEasy\nrEasyString\nrErrorMsg\nrHelpful\nrInterest\nrOverall\nrOverallString\nrStatus\nrTextBookUse\nrWouldTakeAgain\nsId\ntakenForCredit\nteacher\nteacherGrade\nteacherRatingTags\nunUsefulGrouping\nusefulGrouping\n'

In [3]:
df_professors = pd.read_json(open('professors.json'))

In [29]:
df_responses = pd.read_csv(open('profData.csv'), header=0)
df_responses

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 45047: character maps to <undefined>

<h3> Create id to gender dict </h3>

In [5]:
names = df_professors['tFname'].values
tids = df_professors['tid'].values

gender_model = gender.Detector(case_sensitive=False)
genders = {}
u = 0

for i,v in enumerate(names):
    name = v.split(' ')[0]
    g = gender_model.get_gender(name)
    if g == 'male' or g == 'mostly_male':
        genders[tids[i]] = 'M'
    elif g == 'female' or g =='mostly_female':
        genders[tids[i]] = 'F'
    elif g == 'unknown' or g == 'andy':
        genders[tids[i]] = 'U'
        u+=1

<h3>Add genders to response dataframe</h3>

In [6]:
df_responses.insert(4,'gender','M')

In [7]:
for k,v in genders.items():
    index = df_responses[df_responses['tid'] == str(k)].index
    df_responses.loc[index,'gender'] = v

<h3> Get sentiment of comments </h3>

In [8]:
def clean_comment(comment): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", comment).split()) 

In [9]:
def get_comment_sentiment(comment): 
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(clean_comment(comment)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive', analysis.sentiment.polarity
    elif analysis.sentiment.polarity == 0: 
        return 'neutral', analysis.sentiment.polarity
    else: 
        return 'negative', analysis.sentiment.polarity

In [10]:
df_responses.insert(18,'sentimentValue',0)
df_responses.insert(18,'sentiment','positive')

In [11]:
comment = df_responses.loc[0,'rComments']
print(comment)
sentiment,polarity = get_comment_sentiment(comment)
print(sentiment)

Although the materials Prof. Aamari gave are really hard, his notes are helpful and his practice exams are also indicative. Overall, I like this professor, and I have learned a lot! By the way, the homework are tough.
negative


In [12]:
for i in df_responses.index:
    comment = df_responses.loc[i,'rComments']
    if(not pd.isna(comment)):
        sentiment,polarity = get_comment_sentiment(comment)
        df_responses.loc[i,'sentiment'] = sentiment
        df_responses.loc[i,'sentimentValue'] = polarity
    else:
        df_responses.loc[i,'sentiment'] = 'N/A'
        df_responses.loc[i,'sentimentValue'] = 0

In [13]:
dropColumns = ['Unnamed: 0', 'rErrorMsg', 'rStatus', 'teacher', 'unUsefulGrouping', 'usefulGrouping']


In [14]:
df_responses.drop(columns=dropColumns,inplace=True)
df = df.drop(columns=['Unnamed: 0', 'easyColor', 'helpColor', 'clarityColor'])

In [9]:
df_responses.to_csv("modifiedProfInfo.csv")

NameError: name 'df_responses' is not defined

In [15]:
df = pd.read_csv("modifiedProfInfo.csv")
df

,Unnamed: 0,Unnamed: 0.1,tid,tLname,tFname,gender,tDept,attendance,helpCount,id,...,rEasyString,rHelpful,rInterest,rOverall,rOverallString,rTextBookUse,rWouldTakeAgain,takenForCredit,teacherGrade,teacherRatingTags
0,0,0,2325936,Aamari,Eddie,M,Mathematics,Not Mandatory,4.0,30218337,...,4.0,4.0,NaN,4.0,4.0,1.0,1.0,1.0,NaN,"['Lecture heavy', 'Caring', 'Lots of homework']"
1,1,1,2325936,Aamari,Eddie,M,Mathematics,Not Mandatory,3.0,29721878,...,3.0,5.0,NaN,5.0,5.0,1.0,1.0,1.0,NaN,"['Lots of homework', 'Caring', 'Hilarious']"
2,2,2,2325936,Aamari,Eddie,M,Mathematics,Not Mandatory,1.0,29701355,...,4.0,3.0,NaN,3.0,3.0,1.0,1.0,1.0,NaN,"['Accessible outside class', 'Lots of homework..."
3,3,3,2325936,Aamari,Eddie,M,Mathematics,Not Mandatory,0.0,29682099,...,4.0,3.0,NaN,3.0,3.0,1.0,0.0,1.0,NaN,[]
4,4,4,2325936,Aamari,Eddie,M,Mathematics,Not Mandatory,8.0,29557088,...,3.0,5.0,NaN,5.0,5.0,1.0,1.0,1.0,Not sure yet,"['Lots of homework', 'Hilarious', 'Caring']"
5,5,5,2325936,Aamari,Eddie,M,Mathematics,Not Mandatory,7.0,29288327,...,2.0,5.0,NaN,5.0,5.0,1.0,1.0,1.0,A,"['Amazing lectures', 'Accessible outside class..."
6,6,6,286439,Abarbanel,Henry,M,Physics,Not Mandatory,0.0,28955886,...,5.0,1.0,NaN,1.0,1.0,0.0,0.0,1.0,NaN,"['Graded by few things', 'Tough grader']"
7,7,7,286439,Abarbanel,Henry,M,Physics,Not Mandatory,0.0,28730130,...,5.0,1.0,NaN,1.0,1.0,0.0,0.0,1.0,B,"['Graded by few things', 'Lecture heavy']"
8,8,8,286439,Abarbanel,Henry,M,Physics,Mandatory,0.0,25895906,...,4.0,1.0,2.0,1.5,1.5,0.0,NaN,1.0,NaN,[]
9,9,9,286439,Abarbanel,Henry,M,Physics,NaN,0.0,20353335,...,1.0,5.0,5.0,5.0,5.0,1.0,NaN,NaN,NaN,[]


In [19]:
def yesNoToInt(str_in):
    if(str_in == "Yes"):
        return 1
    elif (str_in == "No"):
        return 0
    return str_in
    

In [24]:
def letterToGPA(str_in):
    gpaDict = {
        'A+': 4.0,
        'A' : 4.0,
        'A-': 3.7,
        'B+': 3.3,
        'B' : 3.0,
        'B-': 2.7,
        'C+': 2.3,
        'C' : 2.0,
        'C-': 1.7,
        'D+': 1.3,
        'D' : 1.0,
        'D-': 0.7,
        'F' : 0.0
    }
    return gpaDict.get(str_in, np.nan)

In [20]:
def interestToInt(str_in):
    if(str_in == "Low"):
        return 1
    elif (str_in == "Meh"):
        return 2
    elif (str_in == "Sorta interested"):
        return 3
    elif (str_in == "Really into it"):
        return 4
    elif (str_in == "It's my life"):
        return 5
    return str_in

In [12]:
def genderToInt(str_in):
    if(str_in == 'M'):
        return 1
    elif(str_in == 'F'):
        return -1
    else:
        return np.nan

In [ ]:
df["rTextBookUse"] = df["rTextBookUse"].apply(yesNoToInt)
df["rWouldTakeAgain"] = df["rWouldTakeAgain"].apply(yesNoToInt)
df["takenForCredit"] = df["takenForCredit"].apply(yesNoToInt)
df["rInterest"] = df["rInterest"].apply(interestToInt)
df["gender"] = df["gender"].apply(genderToInt)
df["teacherGrade"] = df["teacherGrade"].apply(letterToGPA)
df

In [25]:
df

,tid,tLname,tFname,gender,tDept,attendance,helpCount,id,notHelpCount,onlineClass,...,rEasyString,rHelpful,rInterest,rOverall,rOverallString,rTextBookUse,rWouldTakeAgain,takenForCredit,teacherGrade,teacherRatingTags
0,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,4.0,30218337,1.0,NaN,...,4.0,4.0,NaN,4.0,4.0,1.0,1.0,1.0,NaN,"['Lecture heavy', 'Caring', 'Lots of homework']"
1,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,3.0,29721878,1.0,NaN,...,3.0,5.0,NaN,5.0,5.0,1.0,1.0,1.0,NaN,"['Lots of homework', 'Caring', 'Hilarious']"
2,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,1.0,29701355,0.0,NaN,...,4.0,3.0,NaN,3.0,3.0,1.0,1.0,1.0,NaN,"['Accessible outside class', 'Lots of homework..."
3,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,0.0,29682099,1.0,NaN,...,4.0,3.0,NaN,3.0,3.0,1.0,0.0,1.0,NaN,[]
4,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,8.0,29557088,4.0,NaN,...,3.0,5.0,NaN,5.0,5.0,1.0,1.0,1.0,Not sure yet,"['Lots of homework', 'Hilarious', 'Caring']"
5,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,7.0,29288327,0.0,NaN,...,2.0,5.0,NaN,5.0,5.0,1.0,1.0,1.0,A,"['Amazing lectures', 'Accessible outside class..."
6,286439,Abarbanel,Henry,1.0,Physics,Not Mandatory,0.0,28955886,0.0,NaN,...,5.0,1.0,NaN,1.0,1.0,0.0,0.0,1.0,NaN,"['Graded by few things', 'Tough grader']"
7,286439,Abarbanel,Henry,1.0,Physics,Not Mandatory,0.0,28730130,1.0,NaN,...,5.0,1.0,NaN,1.0,1.0,0.0,0.0,1.0,B,"['Graded by few things', 'Lecture heavy']"
8,286439,Abarbanel,Henry,1.0,Physics,Mandatory,0.0,25895906,1.0,NaN,...,4.0,1.0,2.0,1.5,1.5,0.0,NaN,1.0,NaN,[]
9,286439,Abarbanel,Henry,1.0,Physics,NaN,0.0,20353335,1.0,NaN,...,1.0,5.0,5.0,5.0,5.0,1.0,NaN,NaN,NaN,[]


In [32]:
df = df.drop(columns=['rOverallString', 'onlineClass'])
df

,tid,tLname,tFname,gender,tDept,attendance,helpCount,id,notHelpCount,quality,...,rEasy,rEasyString,rHelpful,rInterest,rOverall,rTextBookUse,rWouldTakeAgain,takenForCredit,teacherGrade,teacherRatingTags
0,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,4.0,30218337,1.0,good,...,4.0,4.0,4.0,NaN,4.0,1.0,1.0,1.0,NaN,"['Lecture heavy', 'Caring', 'Lots of homework']"
1,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,3.0,29721878,1.0,awesome,...,3.0,3.0,5.0,NaN,5.0,1.0,1.0,1.0,NaN,"['Lots of homework', 'Caring', 'Hilarious']"
2,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,1.0,29701355,0.0,average,...,4.0,4.0,3.0,NaN,3.0,1.0,1.0,1.0,NaN,"['Accessible outside class', 'Lots of homework..."
3,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,0.0,29682099,1.0,average,...,4.0,4.0,3.0,NaN,3.0,1.0,0.0,1.0,NaN,[]
4,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,8.0,29557088,4.0,awesome,...,3.0,3.0,5.0,NaN,5.0,1.0,1.0,1.0,NaN,"['Lots of homework', 'Hilarious', 'Caring']"
5,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,7.0,29288327,0.0,awesome,...,2.0,2.0,5.0,NaN,5.0,1.0,1.0,1.0,4.0,"['Amazing lectures', 'Accessible outside class..."
6,286439,Abarbanel,Henry,1.0,Physics,Not Mandatory,0.0,28955886,0.0,awful,...,5.0,5.0,1.0,NaN,1.0,0.0,0.0,1.0,NaN,"['Graded by few things', 'Tough grader']"
7,286439,Abarbanel,Henry,1.0,Physics,Not Mandatory,0.0,28730130,1.0,awful,...,5.0,5.0,1.0,NaN,1.0,0.0,0.0,1.0,3.0,"['Graded by few things', 'Lecture heavy']"
8,286439,Abarbanel,Henry,1.0,Physics,Mandatory,0.0,25895906,1.0,poor,...,4.0,4.0,1.0,2.0,1.5,0.0,NaN,1.0,NaN,[]
9,286439,Abarbanel,Henry,1.0,Physics,NaN,0.0,20353335,1.0,awesome,...,1.0,1.0,5.0,5.0,5.0,1.0,NaN,NaN,NaN,[]


In [33]:
df.to_csv("modifiedProfInfo.csv", index=False)